In [1]:
import sys
from pathlib import Path
project_root = Path.cwd().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))


# Генетическая стратификация

Этот ноутбук показывает, как использовать взаимозаменяемые алгоритмы **GenRest**.
Мы создадим синтетические данные с категориальными и числовыми
признаками, преобразуем числовой признак в категории, а затем
запустим `GeneticStratificationAlgorithm`.

In [2]:
import pandas as pd
import numpy as np
from genrest import (
    GeneticStratificationAlgorithm,
    InheritedGeneticStratificationAlgorithm,
    bin_numeric,
)

rng = np.random.default_rng(0)
colors = rng.choice(['red', 'blue', 'green'], size=200)
shapes = rng.choice(['circle', 'square', 'triangle'], size=200)
age = rng.normal(30, 10, size=200)
y = ((colors == 'red').astype(int) + (shapes == 'circle').astype(int) + age / 10 + rng.normal(0, 1, 200))
data = pd.DataFrame({'color': colors, 'shape': shapes, 'age': age, 'y': y})
data.head()

,color,shape,age,y
0,green,circle,24.143422,2.318870
1,blue,square,25.274123,2.889915
2,blue,circle,35.863373,5.030329
3,red,circle,23.364648,3.976025
4,red,triangle,23.865822,3.970116


Преобразуем числовой столбец `age` в категории с помощью функции `bin_numeric` и посмотрим на результат.

In [3]:
bin_numeric(data, 'age', bins=3)
data.head()

,color,shape,age,y
0,green,circle,"(-7.724, 24.159]",2.318870
1,blue,square,"(24.159, 32.989]",2.889915
2,blue,circle,"(32.989, 60.66]",5.030329
3,red,circle,"(-7.724, 24.159]",3.976025
4,red,triangle,"(-7.724, 24.159]",3.970116


Теперь запустим генетический алгоритм. В качестве параметра `n_groups` укажем 2, чтобы получить 2^3=8 страт.

In [4]:
stratifier = GeneticStratificationAlgorithm(
    strat_columns=['color', 'shape', 'age'],
    target_col='y',
    population_size=10,
    generations=5,
    n_groups=2,
    random_state=0,
)
best = stratifier.fit(data)
best, stratifier.best_score_

(Stratification(boundaries={'color': {'blue': 1, 'green': 1, 'red': 0}, 'shape': {'circle': 0, 'square': 1, 'triangle': 1}, 'age': {'(-7.724, 24.159]': 0, '(24.159, 32.989]': 1, '(32.989, 60.66]': 1}}),
 0.26765792014745565)

### Преобразование признаков в метку страты

После обучения можно заменить несколько категориальных признаков одной колонкой с описанием полученной страты.

In [5]:
collapsed = stratifier.transform(data, column_name='strata_label')
collapsed.head()


Результаты трансформации:
  Цельная дисперсия: 2.536092
  Стратифицированная дисперсия по исходным комбинациям: 0.054331
  Стратифицированная дисперсия после объединения категорий: 0.267658
  Стратифицированная дисперсия по сравнению с исходной: понижения нет (рост на 392.64% относительно исходной стратифицированной дисперсии)


,y,strata_label
0,2.318870,"strata_4: color={blue, green}; shape={circle};..."
1,2.889915,"strata_7: color={blue, green}; shape={square, ..."
2,5.030329,"strata_5: color={blue, green}; shape={circle};..."
3,3.976025,strata_0: color={red}; shape={circle}; age={(-...
4,3.970116,"strata_2: color={red}; shape={square, triangle..."


## Наследуемые колонки
`InheritedGeneticStratificationAlgorithm` позволяет указать признаки, которые нельзя смешивать между стратами.
Алгоритм совместим с базовым, а результаты можно комбинировать в одном пайплайне.

In [6]:
inherited_algo = InheritedGeneticStratificationAlgorithm(
    strat_columns=['color', 'shape', 'age'],
    target_col='y',
    mandatory_columns=['color'],
    n_groups=2,
    generations=5,
    population_size=10,
    random_state=0,
)
inherited_algo.fit(data)
inherited_algo.transform_to_indices(data)[:5]

array([ 1,  7,  5,  8, 10])

### Преобразование с описанием страт
После обучения можно получить датафрейм с метками, где сочетаются обязательные и объединённые признаки.

In [7]:
inherited_view = inherited_algo.transform(
    data,
    column_name='inherited_strata',
    drop_original=False,
)
inherited_view.head()


Результаты трансформации с обязательными колонками:
  Цельная дисперсия: 2.536092
  Стратифицированная дисперсия по исходным комбинациям: 0.054331
  Стратифицированная дисперсия после объединения категорий: 0.139690
  Стратифицированная дисперсия по сравнению с исходной: понижения нет (рост на 157.11% относительно исходной стратифицированной дисперсии)


,color,shape,age,y,inherited_strata
0,green,circle,"(-7.724, 24.159]",2.318870,color=green; strata_1: shape={circle}; age={(-...
1,blue,square,"(24.159, 32.989]",2.889915,"color=blue; strata_7: shape={square, triangle}..."
2,blue,circle,"(32.989, 60.66]",5.030329,color=blue; strata_5: shape={circle}; age={(24...
3,red,circle,"(-7.724, 24.159]",3.976025,color=red; strata_8: shape={circle}; age={(-7....
4,red,triangle,"(-7.724, 24.159]",3.970116,"color=red; strata_10: shape={square, triangle}..."


### Быстрый запуск через `fit_transform`
Если не нужно сохранять объект, используйте сокращённый вызов, который сразу возвращает преобразованные данные.

In [8]:
quick_view = InheritedGeneticStratificationAlgorithm(
    strat_columns=['color', 'shape', 'age'],
    target_col='y',
    mandatory_columns=['color'],
    n_groups=2,
    generations=5,
    population_size=10,
    random_state=1,
).fit_transform(
    data,
    column_name='quick_strata',
    drop_original=True,
)
quick_view.head()


Результаты трансформации с обязательными колонками:
  Цельная дисперсия: 2.536092
  Стратифицированная дисперсия по исходным комбинациям: 0.054331
  Стратифицированная дисперсия после объединения категорий: 0.139690
  Стратифицированная дисперсия по сравнению с исходной: понижения нет (рост на 157.11% относительно исходной стратифицированной дисперсии)


,y,quick_strata
0,2.318870,color=green; strata_0: shape={circle}; age={(-...
1,2.889915,"color=blue; strata_6: shape={square, triangle}..."
2,5.030329,color=blue; strata_4: shape={circle}; age={(24...
3,3.976025,color=red; strata_9: shape={circle}; age={(-7....
4,3.970116,"color=red; strata_11: shape={square, triangle}..."


## Стратификация по числовому признаку
`bin_numeric` превращает вещественные данные в категории перед запуском алгоритма.


In [9]:
df = pd.DataFrame({'x': rng.normal(size=20), 'y': rng.normal(size=20)})
bin_numeric(df, 'x', bins=4)
GeneticStratificationAlgorithm(['x'], 'y', population_size=5, generations=5, n_groups=2, random_state=0).fit(df)

Stratification(boundaries={'x': {'(-0.245, 0.225]': 1, '(-1.203, -0.245]': 1, '(-2.423, -1.203]': 1, '(0.225, 2.278]': 0}})